# CS211: Data Privacy
## Homework 9

In [ ]:
# Load the data and libraries
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

def laplace_mech(v, sensitivity, epsilon):
    return v + np.random.laplace(loc=0, scale=sensitivity / epsilon)

def gaussian_mech(v, sensitivity, epsilon, delta):
    return v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)

def gaussian_mech_vec(vec, sensitivity, epsilon, delta):
    return [v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)
            for v in vec]

def pct_error(orig, priv):
    return np.abs(orig - priv)/orig * 100.0

# adult = pd.read_csv('https://github.com/jnear/cs211-data-privacy/raw/master/homework/adult_with_pii.csv')

In [ ]:
# Load data files
import numpy as np
import urllib.request
import io

url_x = 'https://github.com/jnear/cs211-data-privacy/raw/master/slides/adult_processed_x.npy'
url_y = 'https://github.com/jnear/cs211-data-privacy/raw/master/slides/adult_processed_y.npy'

with urllib.request.urlopen(url_x) as url:
    f = io.BytesIO(url.read())
X = np.load(f)

with urllib.request.urlopen(url_y) as url:
    f = io.BytesIO(url.read())
y = np.load(f)

In [ ]:
# Split data into training and test sets
training_size = int(X.shape[0] * 0.8)

X_train = X[:training_size]
X_test = X[training_size:]

y_train = y[:training_size]
y_test = y[training_size:]

print('Train and test set sizes:', len(y_train), len(y_test))

## Useful Functions

In [ ]:
# The loss function measures how good our model is. The training goal is to minimize the loss.
# This is the logistic loss function.
def loss(theta, xi, yi):
    exponent = - yi * (xi.dot(theta))
    return np.log(1 + np.exp(exponent))

# This is the gradient of the logistic loss
# The gradient is a vector that indicates the rate of change of the loss in each direction
def gradient(theta, xi, yi):
    exponent = yi * (xi.dot(theta))
    return - (yi*xi) / (1+np.exp(exponent))

def avg_grad(theta, X, y):
    grads = [gradient(theta, xi, yi) for xi, yi in zip(X, y)]
    return np.mean(grads, axis=0)

# Prediction: take a model (theta) and a single example (xi) and return its predicted label
def predict(xi, theta, bias=0):
    label = np.sign(xi @ theta + bias)
    return label

def accuracy(theta):
    return np.sum(predict(X_test, theta) == y_test)/X_test.shape[0]

# L2 Clipping
def L2_clip(v, b):
    norm = np.linalg.norm(v, ord=2)
    
    if norm > b:
        return b * (v / norm)
    else:
        return v

def gradient_sum(theta, X, y, b):
    gradients = [L2_clip(gradient(theta, x_i, y_i), b) for x_i, y_i in zip(X,y)]
        
    # sum query
    # L2 sensitivity is b (by clipping performed above)
    return np.sum(gradients, axis=0)
    
# Noisy gradient descent
# Satisfies (k*epsilon + epsilon, k*delta)-differential privacy
def noisy_gradient_descent(iterations, epsilon, delta):
    theta = np.zeros(X_train.shape[1])
    b = 3

    noisy_count = laplace_mech(X_train.shape[0], 1, epsilon)

    for i in range(iterations):
        clipped_gradient_sum = gradient_sum(theta, X_train, y_train, b)
        noisy_gradient_sum = np.array(gaussian_mech_vec(clipped_gradient_sum, b, epsilon, delta))
        noisy_avg_gradient = noisy_gradient_sum / noisy_count
        theta = theta - noisy_avg_gradient

    return theta

## Question 1 (20 points)

Implement `noisy_gradient_descent_RDP`, a variant of noisy gradient descent that uses Rényi differential privacy. Your solution should have a **total** privacy cost of $(\alpha, \bar\epsilon)$-RDP.

See [Chapter 8](https://uvm-plaid.github.io/programming-dp/notebooks/ch8.html#renyi-differential-privacy) for more information.

In [ ]:
def gaussian_mech_RDP_vec(vec, sensitivity, alpha, epsilon):
    sigma = np.sqrt((sensitivity**2 * alpha) / (2 * epsilon))
    return [v + np.random.normal(loc=0, scale=sigma) for v in vec]

def noisy_gradient_descent_RDP(iterations, alpha, epsilon_bar):
    # YOUR CODE HERE
    raise NotImplementedError()

theta = noisy_gradient_descent_RDP(10, 20, 0.1)
print('Final accuracy:', accuracy(theta))

In [ ]:
# TEST CASE

assert accuracy(noisy_gradient_descent_RDP(5, 100, 0.0001)) > 0.70
assert accuracy(noisy_gradient_descent_RDP(5, 100, 0.00000001)) < 0.75
assert accuracy(noisy_gradient_descent_RDP(5, 10000, 0.00001)) < 0.75

## Question 2 (20 points)

Implement `noisy_gradient_descent_zCDP`, a variant of noisy gradient descent that uses zero-concentrated differential privacy. Your solution should have a **total** privacy cost of $\rho$-zCDP.

See [Chapter 8](https://uvm-plaid.github.io/programming-dp/notebooks/ch8.html#zero-concentrated-differential-privacy) for more information.

In [ ]:
def gaussian_mech_zCDP_vec(vec, sensitivity, rho):
    sigma = np.sqrt((sensitivity**2) / (2 * rho))
    return [v + np.random.normal(loc=0, scale=sigma) for v in vec]

def noisy_gradient_descent_zCDP(iterations, rho):
    # YOUR CODE HERE
    raise NotImplementedError()

theta = noisy_gradient_descent_zCDP(10, 0.1)
print('Final accuracy:', accuracy(theta))

In [ ]:
# TEST CASE

assert accuracy(noisy_gradient_descent_zCDP(5, 0.0000000001)) < 0.75
assert accuracy(noisy_gradient_descent_zCDP(5, 0.0001)) > 0.70

## Question 3 (10 points)

Which of the following functions is likely to produce the best accuracy for a given privacy cost, and why? Which is likely to produce the worst accuracy for a given privacy cost, and why?

- `noisy_gradient_descent`
- `noisy_gradient_descent_RDP`
- `noisy_gradient_descent_zCDP`

YOUR ANSWER HERE

## Question 4 (20 points)

Implement `noisy_gradient_descent_zCDP_ED`, which has a **total privacy cost** of $(\epsilon, \delta)$-differential privacy, but which uses **zero-concentrated differential privacy** to perform composition.

*Hint*: One approach is to:
1. Convert $\epsilon$ and $\delta$ into an equivalent $\rho$ parameter for zCDP
2. Call `noisy_gradient_descent_zCDP` with the $\rho$ from (1)

See [Chapter 8](https://uvm-plaid.github.io/programming-dp/notebooks/ch8.html#zero-concentrated-differential-privacy) for more information on converting between variants.

Note that you will need to *approximate $\rho$ computationally*. In other words, there is no simple mathematical expression to calculate $\rho$ based on $\epsilon$ and $\delta$; instead, you will want to:

1. *Guess* a value for $\rho$
2. Calculate the corresponding $\epsilon$ given $\delta$
3. If $\epsilon$ is larger than the target, decrease the guess for $\rho$. If $\epsilon$ is smaller than the target, increase the guess for $\rho$.
4. Repeat these steps until you find a value for $\rho$ that yields a close approximation of the target for $\epsilon$, but does not exceed it.

In [ ]:
def noisy_gradient_descent_zCDP_ED(iterations, epsilon, delta):
    # YOUR CODE HERE
    raise NotImplementedError()

theta = noisy_gradient_descent_zCDP_ED(10, 1.0, 1e-5)
print('Final accuracy:', accuracy(theta))

In [ ]:
# TEST CASE

assert accuracy(noisy_gradient_descent_zCDP_ED(5, 0.1, 1e-5)) > 0.70
assert accuracy(noisy_gradient_descent_zCDP_ED(5, 1.0, 1e-5)) > 0.70